In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import copy as cp
import scipy as sp
from scipy import misc as sc
%matplotlib inline
atom_position_list = []

##Nucleon Class
The class creates a nucleon object and has a function to generate its location within the atom. 

In [22]:
class Nucleon(object):
    def __init__(self,x_pos,velx,leng_atom,height_change,skin_depth,repulsion_d):
        self.pos = [x_pos,0]
        self.vel = [velx,0]
        self.ncoll = 0
        self.leng_atom = leng_atom
        self.skin_depth = skin_depth
        self.repulsion_d = repulsion_d
        self.height_change = height_change
        
    def createnucleon(self):
        #should_continue = False
        ws = Wood_saxon(self.leng_atom/2, self.skin_depth, 0, 5*self.skin_depth+self.leng_atom/2)

        # The creation of the nucleons for the atom
        looking_iterations = 0 
        pos_found = False
        while pos_found == False:
            pos_found = True
            y_location = ws.rand_woodsaxon()
            #print y_location
            temp_var = np.random.uniform(-1,1)
            if temp_var>0:
                y_location = y_location*-1
            y_location + self.height_change
            looking_iterations += 1
            if self.repulsion_d >0:
                for t in range(len(atom_position_list)):
                    if abs(y_location -atom_position_list[t])< self.repulsion_d:
                        pos_found = False
                        break

            # Appending the values for the location and velocity of the nucleons to the arrays 
        atom_position_list.append(y_location)
        self.pos[1] = y_location

##Atom class
This class creates an atom object which will have a number of nucleons, a velocity, a length, a skin depth, and a repulsion distance. 

In [23]:
class Atom(object):
    def __init__(self):
        self.nucleons = []
    
    def createatom(self,size = 0, x_pos = 0, velx = 0, leng_atom = 0, height_change = 0, skin_depth = 1, repulsion_d = 0):
        for i in range(size):
            nucleonous = Nucleon(x_pos,velx,leng_atom,height_change,skin_depth,repulsion_d)
            nucleonous.createnucleon()
            self.nucleons.append(nucleonous)
            
        global atom_position_list
        atom_position_list = []
        
    def printloc(self):
        for i in range(len(self.nucleons)):
            print self.nucleons[i].pos[1]
    
    def plot_atom(self, x_min = -200, x_max = 200, y_min = -200, y_max = 200):
        
        for i in range(len(self.nucleons)):
            plt.gcf().gca().add_artist( plt.Circle( (self.nucleons[i].pos[0],self.nucleons[i].pos[1]), radius,color='r' ) )
            plt.gcf().gca().add_artist( plt.Circle( (self.nucleons[i].pos[0],self.nucleons[i].pos[1]), radius,color='g', fill = False ) )
            if self.nucleons[i].ncoll > 0:
                plt.quiver(self.nucleons[i].pos[0],self.nucleons[i].pos[1], self.nucleons[i].vel[0],self.nucleons[i].vel[1],angles='xy',scale=214,color='g')
            else:
                plt.quiver(self.nucleons[i].pos[0],self.nucleons[i].pos[1],self.nucleons[i].vel[0],self.nucleons[i].vel[1],angles='xy',scale=214,color='m')
        plt.xlim(x_min,x_max)
        plt.ylim(y_min,y_max)
    

##Collision Class 
This class creates a collision object that can simulate a collision between two intaken atoms. The class can run a collision between two nucleons or a collision between two atoms. It can also run many atom collisions and graph collected data. 

In [3]:
class Collide(object):
    def __init__(self, atom1,atom2):
        self.atom1 = atom1
        self.atom2 = atom2
        self.angles1_temp = []
        self.vx1_temp = []
        self.vy1_temp = []
        self.y1_temp = []
        self.angles2_temp = []
        self.vx2_temp = []
        self.vy2_temp = []
        self.y2_temp = []
        self.skin_depth = 1
        self.inpart = []
        self.incoll =[]
        self.impact_parameters = []
        self.simple_skin_depth = 1
        self.simple_inpart = []
        self.simple_incoll =[]
        self.simple_impact_parameters = []
        self.simple_ideal_multiplicities = []
        self.ideal_multiplicities = []
        self.simple_y1_temp =[]
        self.simple_y2_temp = []
                
    def collision(self,atom1 = [], atom2 = [],Numberinatom1 = 0, Numberinatom2 = 0):
        #initial positions and velocities of the nucleons
        x_position1 = atom1.nucleons[Numberinatom1].pos[0]
        y_position1 = atom1.nucleons[Numberinatom1].pos[1]
        x_velocity1 = atom1.nucleons[Numberinatom1].vel[0]
        y_velocity1 = atom1.nucleons[Numberinatom1].vel[1]

        x_position2 = atom2.nucleons[Numberinatom2].pos[0]
        y_position2 = atom2.nucleons[Numberinatom2].pos[1]
        x_velocity2 = atom2.nucleons[Numberinatom2].vel[0]
        y_velocity2 = atom2.nucleons[Numberinatom2].vel[1]


        #Velocity vector for nucleon 1 and 2
        velocity_vec1 = [x_velocity1, y_velocity1,0] 
        velocity_vec2 = [x_velocity2, y_velocity2,0] 

        #Vector between atom centers
        between_vector = [x_position2-x_position1, y_position2-y_position1]

        #Unit vector for between_vector
        mag_between_vector = np.linalg.norm(between_vector)
        unit_between_vector = [between_vector[0]/mag_between_vector,between_vector[1]/mag_between_vector,0]

        #Dot-product of the vectors with the inbetween vector
        dot_vector1= np.dot(velocity_vec1,unit_between_vector)
        dot_vector2= np.dot(velocity_vec2,unit_between_vector)

        #The projected vectors in the direction of the other atom
        vec1_in_dir = [unit_between_vector[0]*dot_vector1,unit_between_vector[1]*dot_vector1]
        vec2_in_dir = [unit_between_vector[0]*dot_vector2,unit_between_vector[1]*dot_vector2]

        #Calculating the perpendicular vector to between_vector and original vector
        temp_original1_bet_vec = np.cross(unit_between_vector,velocity_vec1)
        temp_original2_bet_vec = np.cross(unit_between_vector,velocity_vec2)

        #Calculating perpendicular vector to temp_original_bet_vec and between_vector
        original1_bet_vec = np.cross(temp_original1_bet_vec, unit_between_vector)
        original2_bet_vec = np.cross(temp_original2_bet_vec, unit_between_vector)

        #magnitude of original_bet_vec
        mag_original1_bet_vec = np.linalg.norm(original1_bet_vec)
        mag_original2_bet_vec = np.linalg.norm(original2_bet_vec)

        #Unit vector for original_bet_vec
        unit_original1_bet_vec = [original1_bet_vec[0]/mag_original1_bet_vec,original1_bet_vec[1]/mag_original1_bet_vec,0]
        unit_original2_bet_vec = [original2_bet_vec[0]/mag_original2_bet_vec,original2_bet_vec[1]/mag_original2_bet_vec,0]

        #The dot product of the unit velocity vector and original_bet_vec
        dot_vector1= np.dot(velocity_vec1,unit_original1_bet_vec)
        dot_vector2= np.dot(velocity_vec2,unit_original2_bet_vec)

        #The projected vectors in the direction away from the other atom
        vec1_in_opp_dir = [unit_original1_bet_vec[0]*dot_vector1,unit_original1_bet_vec[1]*dot_vector1]
        vec2_in_opp_dir = [unit_original2_bet_vec[0]*dot_vector2,unit_original2_bet_vec[1]*dot_vector2]

        #Switching velocities into the x-y-z space
        Vx1 = vec2_in_dir[0] + vec1_in_opp_dir[0]
        Vx2 = vec1_in_dir[0] + vec2_in_opp_dir[0]
        Vy1 = vec2_in_dir[1] + vec1_in_opp_dir[1]
        Vy2 = vec1_in_dir[1] + vec2_in_opp_dir[1]

        return Vx1,Vy1,Vx2,Vy2
        
    
    def collisionsimulation(self, impact_param = 0,atom_first = None, atom_second = None, print_state = 0,multi_coll = True,x_min = -300, x_max = 300, y_min = -300, y_max = 300):
        
        if atom_first == None or atom_second == None:
            atom3 = cp.deepcopy(self.atom2)
            atom1 = cp.deepcopy(self.atom1)
            if impact_param != 0:
                atom2 = y_update(atom3 = atom3, y_change = impact_param)
            else:
                atom2 = atom3
        else:
            atom3 = cp.deepcopy(atom_second)
            atom1 = cp.deepcopy(atom_first)
            if impact_param != 0:
                atom2 = y_update(atom3 = atom3, y_change = impact_param)
            else:
                atom2 = atom3
        
        bx_vel1 = []
        by_vel1 = []
        bx_vel2 = []
        by_vel2 = []
            
        #for i in range(len(atom1.nucleons)):
        #    bx_vel1.append(atom1.nucleons[i].vel[0])
        #    by_vel1.append(atom1.nucleons[i].vel[1])
            
        #for i in range(len(atom2.nucleons)):
        #    bx_vel2.append(atom2.nucleons[i].vel[0])
        #    by_vel2.append(atom2.nucleons[i].vel[1])
        
        #print "Sum of x velocities before ",(sum(bx_vel1) + sum(bx_vel2))
        #print "Sum of y velocities before ",(sum(by_vel1) + sum(by_vel2))
        
        incoll = 0

        if print_state == 1:
            plt.figure( figsize=(15, 5))
            plt.subplot( 131 )
            for i in range(len(atom1.nucleons)):
                    plt.gcf().gca().add_artist( plt.Circle( (atom1.nucleons[i].pos[0],atom1.nucleons[i].pos[1]), radius,color='r' ) )
                    plt.gcf().gca().add_artist( plt.Circle( (atom1.nucleons[i].pos[0],atom1.nucleons[i].pos[1]), radius,color='g', fill = False ) )
                    if atom1.nucleons[i].ncoll > 0:
                        plt.quiver(atom1.nucleons[i].pos[0],atom1.nucleons[i].pos[1], atom1.nucleons[i].vel[0],atom1.nucleons[i].vel[1],angles='xy',scale=214,color='g')
                    else:
                        plt.quiver(atom1.nucleons[i].pos[0],atom1.nucleons[i].pos[1],atom1.nucleons[i].vel[0],atom1.nucleons[i].vel[1],angles='xy',scale=214,color='m')
            for i in range(len(atom2.nucleons)):
                plt.gcf().gca().add_artist( plt.Circle( (atom2.nucleons[i].pos[0],atom2.nucleons[i].pos[1]), radius,color='b' ) )
                plt.gcf().gca().add_artist( plt.Circle( (atom2.nucleons[i].pos[0],atom2.nucleons[i].pos[1]), radius,color='g', fill = False ) )
                if atom2.nucleons[i].ncoll > 0:
                    plt.quiver(atom2.nucleons[i].pos[0],atom2.nucleons[i].pos[1],atom2.nucleons[i].vel[0],atom2.nucleons[i].vel[1],angles='xy',scale=214,color='g')
                else:
                    plt.quiver(atom2.nucleons[i].pos[0],atom2.nucleons[i].pos[1],atom2.nucleons[i].vel[0],atom2.nucleons[i].vel[1],angles='xy',scale=214,color='m')
            plt.xlim(x_min,x_max)
            plt.ylim(y_min,y_max)

        graph = 0

        for k in range(num_iterations):


            #running through the particles and updating their locations
            for i in range(len(atom1.nucleons)):
                atom1.nucleons[i].pos[0] = atom1.nucleons[i].pos[0] + atom1.nucleons[i].vel[0] * tinc
                atom1.nucleons[i].pos[1] = atom1.nucleons[i].pos[1] + atom1.nucleons[i].vel[1] * tinc

            for i in range(len(atom2.nucleons)):
                atom2.nucleons[i].pos[0] = atom2.nucleons[i].pos[0] + atom2.nucleons[i].vel[0] * tinc
                atom2.nucleons[i].pos[1] = atom2.nucleons[i].pos[1] + atom2.nucleons[i].vel[1] * tinc

            for i in range(len(atom1.nucleons)):
                for k in range(len(atom2.nucleons)):
                    if (atom1.nucleons[i].ncoll == 0 and atom2.nucleons[k].ncoll == 0) or multi_coll:
                        distance = np.sqrt((atom1.nucleons[i].pos[1]-atom2.nucleons[k].pos[1])**2 + (atom2.nucleons[k].pos[0]-atom1.nucleons[i].pos[0])**2)
                        if distance <= 2*radius:
                            temp_var = self.collision(atom1 = atom1, atom2 = atom2,Numberinatom1 = i, Numberinatom2 = k)
                            incoll += 1
                            atom1.nucleons[i].vel[0] = temp_var[0]
                            atom1.nucleons[i].vel[1]  = temp_var[1]
                            atom2.nucleons[k].vel[0] = temp_var[2]
                            atom2.nucleons[k].vel[1] = temp_var[3]
                            atom1.nucleons[i].ncoll += 1
                            atom2.nucleons[k].ncoll += 1
                            if print_state == 1 and graph ==0:
                                plt.subplot( 132 )
                                for i in range(len(atom1.nucleons)):
                                        plt.gcf().gca().add_artist( plt.Circle( (atom1.nucleons[i].pos[0],atom1.nucleons[i].pos[1]), radius,color='r' ) )
                                        plt.gcf().gca().add_artist( plt.Circle( (atom1.nucleons[i].pos[0],atom1.nucleons[i].pos[1]), radius,color='g', fill = False ) )
                                for i in range(len(atom2.nucleons)):
                                    plt.gcf().gca().add_artist( plt.Circle( (atom2.nucleons[i].pos[0],atom2.nucleons[i].pos[1]), radius,color='b' ) )
                                    plt.gcf().gca().add_artist( plt.Circle( (atom2.nucleons[i].pos[0],atom2.nucleons[i].pos[1]), radius,color='g', fill = False ) )
                                plt.xlim(x_min,x_max)
                                plt.ylim(y_min,y_max)
                                graph = 1

        if print_state == 1:
            plt.subplot( 133 )
            for i in range(len(atom1.nucleons)):
                    plt.gcf().gca().add_artist( plt.Circle( (atom1.nucleons[i].pos[0],atom1.nucleons[i].pos[1]), radius,color='r' ) )
                    plt.gcf().gca().add_artist( plt.Circle( (atom1.nucleons[i].pos[0],atom1.nucleons[i].pos[1]), radius,color='g', fill = False ) )
                    if atom1.nucleons[i].ncoll > 0:
                        plt.quiver(atom1.nucleons[i].pos[0],atom1.nucleons[i].pos[1], atom1.nucleons[i].vel[0],atom1.nucleons[i].vel[1],angles='xy',scale=214,color='g')
                    else:
                        plt.quiver(atom1.nucleons[i].pos[0],atom1.nucleons[i].pos[1],atom1.nucleons[i].vel[0],atom1.nucleons[i].vel[1],angles='xy',scale=214,color='m')
            for i in range(len(atom2.nucleons)):
                plt.gcf().gca().add_artist( plt.Circle( (atom2.nucleons[i].pos[0],atom2.nucleons[i].pos[1]), radius,color='b' ) )
                plt.gcf().gca().add_artist( plt.Circle( (atom2.nucleons[i].pos[0],atom2.nucleons[i].pos[1]), radius,color='g', fill = False ) )
                if atom2.nucleons[i].ncoll > 0:
                    plt.quiver(atom2.nucleons[i].pos[0],atom2.nucleons[i].pos[1],atom2.nucleons[i].vel[0],atom2.nucleons[i].vel[1],angles='xy',scale=214,color='g')
                else:
                    plt.quiver(atom2.nucleons[i].pos[0],atom2.nucleons[i].pos[1],atom2.nucleons[i].vel[0],atom2.nucleons[i].vel[1],angles='xy',scale=214,color='m')
            plt.xlim(x_min,x_max)
            plt.ylim(y_min,y_max)

        inpart = 0

        for i in range(len(atom1.nucleons)):
            if atom1.nucleons[i].ncoll > 0:
                inpart += 1

        for i in range(len(atom2.nucleons)):
            if atom2.nucleons[i].ncoll >0:
                inpart += 1

        angle1 = []
        angle2 = []


        for k in range(len(atom1.nucleons)):
            temp_val = car_to_polar(vector_x = atom1.nucleons[k].vel[0], vector_y = atom1.nucleons[k].vel[1])
            angle1.append(temp_val[1])

        for k in range(len(atom2.nucleons)):
            temp_val = car_to_polar(vector_x = atom2.nucleons[k].vel[0], vector_y = atom2.nucleons[k].vel[1])
            angle2.append(temp_val[1])
        
        x_vel1 = []
        y_vel1 = []
        x_vel2 = []
        y_vel2 = []
            
        for i in range(len(atom1.nucleons)):
            x_vel1.append(atom1.nucleons[i].vel[0])
            y_vel1.append(atom1.nucleons[i].vel[1])
            
        for i in range(len(atom2.nucleons)):
            x_vel2.append(atom2.nucleons[i].vel[0])
            y_vel2.append(atom2.nucleons[i].vel[1])
        
        #print "Sum of x velocities after ",(sum(x_vel1) + sum(x_vel2))
        #print "Sum of y velocities after ",(sum(y_vel1) + sum(y_vel2))
        
        #print "Number of participants ", inpart
        #print "Number of collisions ", incoll

        return angle1,angle2,impact_param, x_vel1, x_vel2, y_vel1, y_vel2,inpart,incoll
    
    def simple_collisionsimulation(self, impact_param = 0,atom_first = None, atom_second = None):
        if atom_first == None or atom_second == None:
            atom3 = cp.deepcopy(self.atom2)
            atom1 = cp.deepcopy(self.atom1)
            if impact_param != 0:
                atom2 = y_update(atom3 = atom3, y_change = impact_param)
            else:
                atom2 = atom3
        else:
            atom3 = cp.deepcopy(atom_second)
            atom1 = cp.deepcopy(atom_first)
            if impact_param != 0:
                atom2 = y_update(atom3 = atom3, y_change = impact_param)
            else:
                atom2 = atom3
        incoll = 0
        
        for i in range(len(atom1.nucleons)):
            for k in range(len(atom2.nucleons)):
                distance = (atom1.nucleons[i].pos[1]-atom2.nucleons[k].pos[1])**2
                if distance <= 2*radius:
                    incoll += 1
                    atom1.nucleons[i].ncoll += 1
                    atom2.nucleons[k].ncoll += 1
        inpart = 0

        for i in range(len(atom1.nucleons)):
            if atom1.nucleons[i].ncoll > 0:
                inpart += 1

        for i in range(len(atom2.nucleons)):
            if atom2.nucleons[i].ncoll >0:
                inpart += 1
        
        return inpart, incoll, impact_param
    
    def simple_many_collisionsimulation(self,num_collisions = 0, min_impactparam = 0, max_impactparam = 0, skin_depth =1, repulsion_d = 0, bw = 50, num_succ = 50, chance_succ = .4, frac_moment = 3, ntrials = 300, res = 1100):
        impact_parameters = []
        inpart = []
        incoll = []
        y1 = []
        y2 = []
        check = True
        self.simple_skin_depth = skin_depth

        for l in range(num_collisions):
            
            atom3 = Atom()
            atom4 = Atom()
            atom3.createatom(size = size1, x_pos = x_location1, velx = velocity1, leng_atom = lengthofatom1, skin_depth = skin_depth, repulsion_d = repulsion_d)
            atom4.createatom(size = size2, x_pos = x_location2, velx = -velocity2, leng_atom = lengthofatom2, skin_depth = skin_depth, repulsion_d = repulsion_d)

            for i in range(len(atom3.nucleons)):
                y1.append(atom3.nucleons[i].pos[1])
            for i in range(len(atom4.nucleons)):
                y2.append(atom4.nucleons[i].pos[1])
                
            if check == True:
                plt.figure( figsize=(15, 5)) 
                plt.subplots_adjust(wspace=.2, hspace=.5)
                
                plt.subplot(121)
                ws1 = Wood_saxon(lengthofatom1/2,self.simple_skin_depth,0,5*self.simple_skin_depth +lengthofatom1/2)
                ws1.complete_wood_saxon()
                plt.hist(y1,bins = (lengthofatom1 / bw), normed=True);
                plt.title("atom1 location of atoms")
                #plt.ylabel('Number of Collisions')
                plt.xlabel('Y-locations')

                plt.subplot( 122 )
                ws2 = Wood_saxon(lengthofatom2/2,self.simple_skin_depth,0,5*self.simple_skin_depth +lengthofatom2/2)
                ws2.complete_wood_saxon()
                plt.hist(y2,bins = (lengthofatom2 / bw), normed=True);
                plt.title("atom2 location of atoms")
                #plt.ylabel('Number of Collisions')
                plt.xlabel('Y-locations')
                
                check = False
                
                
            impact_pam = np.random.uniform(min_impactparam,max_impactparam)


            temp_var = self.simple_collisionsimulation(impact_param = impact_pam, atom_first = atom3, atom_second= atom4)
            inpart.append(temp_var[0])
            incoll.append(temp_var[1])
            impact_parameters.append(temp_var[2])

        self.simple_y1_temp = y1
        self.simple_y2_temp = y2
        self.simple_inpart = inpart
        self.simple_incoll = incoll
        self.simple_impact_parameters = impact_parameters
        
        nbd_var = Neg_binom_dist(num_succ,chance_succ,frac_moment)    
        for i in range(len(self.simple_inpart)):
            if self.simple_inpart[i] != 0:
                temp_var = nbd_var.get_idealmultiplicity(inpart = self.simple_inpart[i], incoll = self.simple_incoll[i], x_max = ntrials, res = res)
                self.simple_ideal_multiplicities.append(temp_var)
    
    def simple_manysimulationsplots(self,nbins = 50,bw = 50):
        
        plt.figure( figsize=(15, 13)) 
        plt.subplots_adjust(wspace=.2, hspace=.5)

        inpart_bins = int(np.amax(self.simple_inpart))
        incoll_bins = int(np.amax(self.simple_incoll))
        H, xedges, yedges = np.histogram2d(self.simple_inpart,self.simple_impact_parameters,bins=(inpart_bins,nbins))
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 321 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("impact parameter vs number of particles")
        plt.ylabel('Number of Participants')
        plt.xlabel('Impact Parameters')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')

        H, xedges, yedges = np.histogram2d(self.simple_incoll,self.simple_impact_parameters,bins=(incoll_bins,nbins))
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 322 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("impact parameter vs number of collisions")
        plt.ylabel('Number of Collisions')
        plt.xlabel('Impact Parameters')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')

        H, xedges, yedges = np.histogram2d(self.simple_incoll,self.simple_inpart,bins=(incoll_bins,inpart_bins))
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 323 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("Number of Collisions vs. Number of Participants")
        plt.ylabel('Number of Collisions')
        plt.xlabel('Number of Participants')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')
        
        plt.subplot( 3,2,4 )
        plt.hist(self.simple_ideal_multiplicities,bins = nbins);
        plt.title("Ideal multiplicities")
        #plt.ylabel('Number of Collisions')
        plt.xlabel('Ideal multiplicities')
        plt.yscale('log')


        plt.subplot( 3,2,5 )
        ws1 = Wood_saxon(lengthofatom1/2,self.simple_skin_depth,0,5*self.simple_skin_depth+lengthofatom1/2)
        ws1.complete_wood_saxon()
        plt.hist(self.simple_y1_temp,bins = (lengthofatom1 / bw), normed=True);
        plt.title("atom1 location of atoms")
        #plt.ylabel('Number of Collisions')
        plt.xlabel('Y-locations')

        plt.subplot( 3,2,6 )
        ws2 = Wood_saxon(lengthofatom2/2,self.simple_skin_depth,0,5*self.simple_skin_depth+lengthofatom2/2)
        ws2.complete_wood_saxon()
        plt.hist(self.simple_y2_temp,bins = (lengthofatom2 / bw), normed=True);
        plt.title("atom2 location of atoms")
        #plt.ylabel('Number of Collisions')
        plt.xlabel('Y-locations')
        
        
        
        
    
    def many_collisionsimulation(self,num_collisions = 0, min_impactparam = 0, max_impactparam = 0, skin_depth =1, repulsion_d = 0, num_succ = 50, chance_succ = .4, frac_moment = 3, ntrials = 300, res = 1100):
        impact_parameters = []
        inpart = []
        incoll = []
        angles1 = []
        angles2 = []
        vx1 = []
        vx2 = []
        vy1 = []
        vy2 = []
        y1 = []
        y2 = []

        for l in range(num_collisions):
            
            atom3 = Atom()
            atom4 = Atom()
            atom3.createatom(size = size1, x_pos = x_location1, velx = velocity1, leng_atom = lengthofatom1, skin_depth = skin_depth, repulsion_d = repulsion_d)
            atom4.createatom(size = size2, x_pos = x_location2, velx = -velocity2, leng_atom = lengthofatom2, skin_depth = skin_depth, repulsion_d = repulsion_d)
            
            for i in range(len(atom3.nucleons)):
                y1.append(atom3.nucleons[i].pos[1])
            for i in range(len(atom4.nucleons)):
                y2.append(atom4.nucleons[i].pos[1])

            impact_pam = np.random.uniform(min_impactparam,max_impactparam)

            #print impact_pam

            temp_var = self.collisionsimulation(impact_param = impact_pam, atom_first = atom3, atom_second= atom4,multi_coll = True)
            impact_parameters.append(temp_var[2])
            angles1.append(temp_var[0])
            angles2.append(temp_var[1])
            vx1.append(temp_var[3])
            vx2.append(temp_var[4])
            vy1.append(temp_var[5])
            vy2.append(temp_var[6])
            inpart.append(temp_var[7])
            incoll.append(temp_var[8])

        #print angles1
        self.angles1_temp = convert_to_1darrays(impactparameters = impact_parameters, secondarray = angles1)
        self.angles2_temp = convert_to_1darrays(impactparameters = impact_parameters, secondarray = angles2)
        self.vx1_temp = convert_to_1darrays(impactparameters = impact_parameters, secondarray = vx1)
        self.vx2_temp = convert_to_1darrays(impactparameters = impact_parameters, secondarray = vx2)
        self.vy1_temp = convert_to_1darrays(impactparameters = impact_parameters, secondarray = vy1)
        self.vy2_temp = convert_to_1darrays(impactparameters = impact_parameters, secondarray = vy2)
        self.y1_temp = y1
        self.y2_temp = y2
        self.skin_depth = skin_depth
        self.inpart = inpart
        self.incoll = incoll
        self.impact_parameters = impact_parameters
        
        nbd_var = Neg_binom_dist(num_succ,chance_succ,frac_moment)    
        for i in range(len(self.inpart)):
            if self.inpart[i] != 0:
                temp_var = nbd_var.get_idealmultiplicity(inpart = self.inpart[i], incoll = self.incoll[i], x_max = ntrials, res = res)
                self.ideal_multiplicities.append(temp_var)
            
        
    def manysimulationsplots(self,nbins = 50,bw = 50):
        
        plt.figure( figsize=(15, 13)) 
        plt.subplots_adjust(wspace=.2, hspace=.5)

        inpart_bins = int(np.amax(self.inpart))
        incoll_bins = int(np.amax(self.incoll))
        H, xedges, yedges = np.histogram2d(self.angles1_temp[0],self.angles1_temp[1],bins=nbins)
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 621 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("atom1 angles")
        plt.xlabel('Angles')
        plt.ylabel('Impact Parameters')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')

        H, xedges, yedges = np.histogram2d(self.angles2_temp[0],self.angles2_temp[1],bins=nbins)
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 622 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("atom2 angles")
        plt.xlabel('Angles')
        plt.ylabel('Impact Parameters')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')

        H, xedges, yedges = np.histogram2d(self.vx1_temp[0],self.vx1_temp[1],bins=nbins)
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 623 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("atom1 x-velocities")
        plt.xlabel('X-Velocity')
        plt.ylabel('Impact Parameters')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')

        H, xedges, yedges = np.histogram2d(self.vx2_temp[0],self.vx2_temp[1],bins=nbins)
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 624 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("atom2 x-velocities")
        plt.xlabel('Angles')
        plt.ylabel('Impact Parameters')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')

        H, xedges, yedges = np.histogram2d(self.vy1_temp[0],self.vy1_temp[1],bins=nbins)
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 625 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("atom1 y-velocities")
        plt.xlabel('Y-Velocites')
        plt.ylabel('Impact Parameters')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')

        H, xedges, yedges = np.histogram2d(self.vy2_temp[0],self.vy2_temp[1],bins=nbins)
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 626 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("atom2 y-velocities")
        plt.xlabel('Y-Velocites')
        plt.ylabel('Impact Parameters')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')

        H, xedges, yedges = np.histogram2d(self.inpart,self.impact_parameters,bins=(inpart_bins,nbins))
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 627 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("impact parameter vs number of particles")
        plt.ylabel('Number of Participants')
        plt.xlabel('Impact Parameters')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')

        H, xedges, yedges = np.histogram2d(self.incoll,self.impact_parameters,bins=(incoll_bins,nbins))
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 628 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("impact parameter vs number of collisions")
        plt.ylabel('Number of Collisions')
        plt.xlabel('Impact Parameters')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')

        H, xedges, yedges = np.histogram2d(self.incoll,self.inpart,bins=(incoll_bins,inpart_bins))
        Hmasked = np.ma.masked_where(H<.05,H)
        plt.subplot( 629 )
        plt.pcolormesh(yedges,xedges,Hmasked)
        plt.title("Number of Collisions vs. Number of Participants")
        plt.ylabel('Number of Collisions')
        plt.xlabel('Number of Participants')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Counts')
        
        plt.subplot( 6,2,10 )
        plt.hist(self.ideal_multiplicities,bins = nbins);
        plt.title("Ideal multiplicities")
        #plt.ylabel('Number of Collisions')
        plt.xlabel('Ideal multiplicities')
        plt.yscale('log')


        plt.subplot( 6,2,11 )
        ws1 = Wood_saxon(lengthofatom1/2,self.skin_depth,0,5*self.skin_depth+lengthofatom1/2)
        ws1.complete_wood_saxon()
        plt.hist(self.y1_temp,bins = (lengthofatom1 / bw), normed=True);
        plt.title("atom1 location of atoms")
        #plt.ylabel('Number of Collisions')
        plt.xlabel('Y-locations')

        plt.subplot( 6,2,12 )
        ws2 = Wood_saxon(lengthofatom2/2,self.skin_depth,0,5*self.skin_depth+lengthofatom2/2)
        ws2.complete_wood_saxon()
        plt.hist(self.y2_temp,bins = (lengthofatom2 / bw), normed=True);
        plt.title("atom2 location of atoms")
        #plt.ylabel('Number of Collisions')
        plt.xlabel('Y-locations')


##Wood_saxon Class
This class creates a wood saxon object with a radius, minimum distance, a maximum distance, and skin depth. Functions within the class include a plotter and a random number generator. 

In [25]:
class Wood_saxon(object):
    def __init__(self, R,a,x_min,x_max):
        self.radius = R
        self.skin_depth = a
        self.x_min = x_min
        self.x_max = x_max
    
    def rand_woodsaxon(self):
        x = np.random.uniform(0,1)
    
        u_max = np.exp(-(self.x_max-self.radius)/float(self.skin_depth))
        u_min = np.exp(-(self.x_min-self.radius)/float(self.skin_depth))
        A = self.skin_depth*(-np.log(u_max+1) + np.log(u_min+1))

        rand_g = np.exp(-x*A/self.skin_depth + np.log(u_min+1)) - 1
        proper_rand_x = -self.skin_depth*np.log(rand_g) + self.radius

        return proper_rand_x
    
    def complete_wood_saxon(self, res = 2000):
        u_max = np.exp(-(self.x_max-self.radius)/float(self.skin_depth))
        u_min = np.exp(-(self.x_min-self.radius)/float(self.skin_depth))
        A = 2*self.skin_depth*(-np.log(u_max+1) + np.log(u_min+1))

        x = np.linspace(self.x_min,self.x_max, res)
        y = 1/(A*(1+np.exp((x-self.radius)/float(self.skin_depth))))
        height = 1/(A*(1+np.exp((0-self.radius)/float(self.skin_depth))))
        plt.plot(x,y)
        plt.plot(-x,y)
        plt.plot([self.radius,self.radius],[0,height])
        plt.plot([self.radius-self.skin_depth/2,self.radius+self.skin_depth/2],[height/2,height/2])
        plt.plot([0,0],[0,height])
        plt.ylim(0,height*1.2)
    

##Negative binomial distribution class
This class creates an nbd object with a number of successes, the chance of success, and the fraction of momentum held by each nucleon within an atom. It can graph the nbd, generate a random value of x based on the probability of an experiment having a number of successes in a certain number of trials. The function calculates the multiplicity as well as the efficiency. 

In [5]:
class Neg_binom_dist(object):
    def __init__(self, num_successes, chance_success, frac_moment):
        self.frac_moment = frac_moment
        self.num_succ = num_successes
        self.prob_of_succ = chance_success
        self.nbins = 0
        self.bin_array = []
        self.x_max = 0
        self.intervals = []
        self.multi2comp = 0
        
    def plot_ntrials(self, ntrials = 40, res = 0):
        if res==0:
            res = ntrials
        x = np.linspace(0,ntrials,res)
        norm_const = self.norm_nbd(res = res, x_max = ntrials)
        y = sc.comb(x-1,self.num_succ -1, exact=False)*(self.prob_of_succ**self.num_succ)*((1-self.prob_of_succ)**(x-self.num_succ))/norm_const
        plt.scatter(x,y)
        plt.xlabel("Number of Trials")
        plt.ylabel("Probability")
        
    def norm_nbd(self, res = 0, x_max = 0):
        x = np.linspace(0,x_max,res)
        y = sc.comb(x-1,self.num_succ -1, exact=False)*(self.prob_of_succ**self.num_succ)*((1-self.prob_of_succ)**(x-self.num_succ)) 
        norm_const = np.trapz(y,dx = x_max/float(res))
        return norm_const
    
        
    def rand_nbd(self,x_max = 0, res = 0):
        
        if (res != self.nbins) or (x_max != self.x_max):
            #self.nbins = res
            norm_const = self.norm_nbd(res = res, x_max = x_max)
            #print norm_const
            inter_dist = x_max/(float(res))
            intervals = np.linspace(0,x_max,res) #maybe
            count = 0
            self.bin_array = []
            self.nbins = res
            self.x_max = x_max
            self.intervals = intervals
            y = sc.comb(intervals-1,self.num_succ -1, exact=False)*(self.prob_of_succ**self.num_succ)*((1-self.prob_of_succ)**(intervals-self.num_succ))/norm_const
            #print y
            area_seg = y * inter_dist
            for i in range(len(area_seg)):
                count += area_seg[i]
                self.bin_array.append(count)

        temp_val_rand = np.random.uniform(0,1)
        final_val_rand = 0
        check = 0
        for i in range(len(self.bin_array) - 1):
            if temp_val_rand > self.bin_array[i] and temp_val_rand <= self.bin_array[i+1]:
                final_val_rand = self.intervals[i]
                check = 1
        if check == 0:
            final_val_rand = self.intervals[-1]
        return final_val_rand 
    
    def get_multi2comp(self, inpart = 0, incoll =0 ):
        if self.frac_moment == 0:
            self.multi2comp = inpart
        else:
            self.multi2comp = (1/2) * ( 1.0 - self.frac_moment ) * inpart + self.frac_moment * incoll
        
    def get_idealmultiplicity(self, inpart = 0, incoll = 0, x_max = 0, res = 1):
        self.get_multi2comp(inpart = inpart, incoll =incoll)
        total_p = 0
        for i in range(int(self.multi2comp)):
            total_p += self.rand_nbd(x_max = x_max, res = res)
        return total_p

    def get_efficiency(self, eff0 = 0):
        return eff0 * ( 1.0 - self.multi2comp * 0.14 / 560 )
            
        
    def print_bins(self):
        print self.bin_array
                
                
                
        
        
        
    
        
    ##Integrate the function here

##Conversion from Cartesian to Polar
This function converts cartesian vectors into polar ones. 

In [10]:
def car_to_polar(vector_x = 0, vector_y = 0):

    #calculating the angle between vector and the horizonal
    theta = np.arctan2(vector_y,vector_x)
    
    #Changing theta to a positive angle
    #if theta < 0:
    #    theta = theta + 2*np.pi
    
    #calculating magnitude of velocity
    magnitude = np.sqrt(vector_x**2 + vector_y**2)
    
    return magnitude,theta

##Updating y coordinates
This function intakes a list of the information for an atom and updates all the y coordinates so that a single atom could be used multiple times, but with their y positions translated updwards or downwards. 

In [34]:
def y_update(atom3 = None, y_change = 0):
    for i in range(len(atom3.nucleons)):
        atom3.nucleons[i].pos[1] += y_change
    
    return atom3

##Converting Cross-Section to Radius
This function intakes the Cross-Section and returns the Radius.

In [68]:
def cross_to_radius(cross_section = 0):
    radius = np.sqrt(cross_section/np.pi)
    return radius

#Collision Simulation
The collisionsimulation function simulates a collision between two atoms. It intakes the impact parameter between the two atoms, the atoms to be collided and whether or not a graph of the collision should be created. Once the atom collision is simulated, the function collects the final angles of the nucleons in each atom, as well as the impact parameter of the whole atom. print_state set to 0 does not print the collision diagrams while set to 1 does. 

## Moving Collision Simualtion Function
This function creates a simulation but updates it in a way that can be used by matplotlib animate

In [70]:
def animated_collisionsimulation(atom1 = None, atom2 = None, multiple_coll = True):
    
    radius = cross_to_radius(cross_section = nucleonxs)

    incoll = 0


    #running through the particles and updating their locations
    for i in range(size1):
        atom1[0][i] = atom1[0][i] + atom1[2][i] * tinc
        atom1[1][i] = atom1[1][i] + atom1[3][i] * tinc

    for i in range(size2):
        atom2[0][i] = atom2[0][i] + atom2[2][i] * tinc
        atom2[1][i] = atom2[1][i] + atom2[3][i] * tinc

    for i in range(size1):
        for k in range(size2):
            if (atom1[5][i] == False and atom2[5][k] == False) or multiple_coll:
                distance = np.sqrt((atom1[1][i]-atom2[1][k])**2 + (atom1[0][i]-atom2[0][k])**2)
                if distance <= 2*radius:
                    temp_var = collision(atom1 = atom1, atom2 = atom2, Numberinatom1 = i, Numberinatom2 = k)
                    incoll += 1
                    atom1[2][i] = temp_var[0]
                    atom1[3][i] = temp_var[1]
                    atom2[2][k] = temp_var[2]
                    atom2[3][k] = temp_var[3]
                    atom1[5][i] = True
                    atom2[5][k] = True

    inpart = 0
    
    for i in range(len(atom1[5])):
        if atom1[5][i] == True:
            inpart += 1
            
    for i in range(len(atom2[5])):
        if atom2[5][i] == True:
            inpart += 1
    
    angle1 = []
    angle2 = []
    
    for k in range(len(atom1[0])):
        temp_val = car_to_polar(vector_x = atom1[2][k], vector_y = atom1[3][k])
        angle1.append(temp_val[1])
        
    for k in range(len(atom2[0])):
        temp_val = car_to_polar(vector_x = atom2[2][k], vector_y = atom2[3][k])
        angle2.append(temp_val[1])
        
    global atom444,atom555
    atom444[0] = atom1[0]
    atom444[1] = atom1[1]
    atom555[0] = atom2[0]
    atom555[1] = atom2[1]
        
    
    return atom1,atom2,inpart,incoll;

In [71]:
def animate(i):
    
    
    global atom444,atom555
    
    radius = cross_to_radius(cross_section = nucleonxs)

    incoll = 0


    #running through the particles and updating their locations
    for i in range(size1):
        atom444[0][i] = atom444[0][i] + atom444[2][i] * tinc
        atom444[1][i] = atom444[1][i] + atom444[3][i] * tinc

    for i in range(size2):
        atom555[0][i] = atom555[0][i] + atom555[2][i] * tinc
        atom555[1][i] = atom555[1][i] + atom555[3][i] * tinc

    for i in range(size1):
        for k in range(size2):
            if (atom444[5][i] == False and atom555[5][k] == False) or multiple_coll:
                distance = np.sqrt((atom444[1][i]-atom555[1][k])**2 + (atom444[0][i]-atom555[0][k])**2)
                if distance <= 2*radius:
                    temp_var = collision(atom1 = atom444, atom2 = atom555, Numberinatom1 = i, Numberinatom2 = k)
                    incoll += 1
                    atom444[2][i] = temp_var[0]
                    atom444[3][i] = temp_var[1]
                    atom555[2][k] = temp_var[2]
                    atom555[3][k] = temp_var[3]
                    atom444[5][i] = True
                    atom555[5][k] = True

    inpart = 0
    
    for i in range(len(atom444[5])):
        if atom444[5][i] == True:
            inpart += 1
            
    for i in range(len(atom555[5])):
        if atom555[5][i] == True:
            inpart += 1
    
    angle1 = []
    angle2 = []
    
    for k in range(len(atom444[0])):
        temp_val = car_to_polar(vector_x = atom444[2][k], vector_y = atom444[3][k])
        angle1.append(temp_val[1])
        
    for k in range(len(atom555[0])):
        temp_val = car_to_polar(vector_x = atom555[2][k], vector_y = atom555[3][k])
        angle2.append(temp_val[1])
    
    
    
    global line
    '''for i in range(max(size1,size2)):
        if i < size1:
            plt.gcf().gca().add_artist( plt.Circle( (temp_var[0][0][i],temp_var[0][1][i]), radius,color='r' ) )
            plt.gcf().gca().add_artist( plt.Circle( (temp_var[0][0][i],temp_var[0][1][i]), radius,color='g', fill = False ) )
        if i < size2:
            plt.gcf().gca().add_artist( plt.Circle( (temp_var[1][0][i],temp_var[1][1][i]), radius,color='b' ) )
            plt.gcf().gca().add_artist( plt.Circle( (temp_var[1][0][i],temp_var[1][1][i]), radius,color='g', fill = False ) )'''
    line.set_data(atom444[0],atom444[1])
    #atom444 = temp_var[0]
    #atom555 = temp_var[1]
    return line

In [72]:
def animated_collision(atom1 = None, atom2 = None, multiple_coll = True,xlow_bound = -300,xhigh_bound = 300,ylow_bound = -300,yhigh_bound =300):
    global atom444, atom555, multiple_coll111, line
    atom444 = cp.deepcopy(atom1)
    atom555 = cp.deepcopy(atom2)
    multiple_col111 =cp.deepcopy(multiple_coll)
    
    #the figure, the axis, and the plot element we want to animate
    fig = plt.figure()
    ax = plt.axes(xlim=(xlow_bound, xhigh_bound), ylim=(ylow_bound, yhigh_bound))
    line, = ax.plot([], [], lw=2)

    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        return line
    
    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init,frames=200, interval=20, blit=True)

    # save the animation as an mp4.  This requires ffmpeg or mencoder to be
    # installed.  The extra_args ensure that the x264 codec is used, so that
    # the video can be embedded in html5.  You may need to adjust this for
    # your system: for more information, see
    # http://matplotlib.sourceforge.net/api/animation_api.html

    plt.show()

In [73]:
"""size1=197
size2=197
velocity1=30
velocity2=-40
x_location1 = -5
x_location2 = 5
atom1 = createatom(size1, 1, 0, velocity1, 200, a = 30)
atom2 = createatom(size2, 2, 0, velocity2, 200, a = 30)

xlow_bound = -300
xhigh_bound = 300
ylow_bound = -300
yhigh_bound = 300

global atom444, atom555, multiple_coll111, line
atom444 = cp.deepcopy(atom1)
atom555 = cp.deepcopy(atom2)

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(xlow_bound, xhigh_bound), ylim=(ylow_bound, yhigh_bound))
line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,frames=200, interval=20, blit=True)

# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html

plt.show()""";

##Many collisions
Many_collisionsimulation creates num_collision number of atom collisions, all varrying in impact parameter between min_impactparam and max_impactparam. The impact parameter in this range is generated randomly. While the specified number of collisions is taking place, the trajectory angles of all the nucleons is collected and eventually graphed out. 

The many collisions function with new atoms created for each time

##One-Dimensional collision plots

In [77]:
def d2_histogram_array(lowestimpact = 0, highestimpact = 0, bins = 0, impact_array = None, second_array = None):
    
    arrayvals = np.linspace(lowestimpact,highestimpact,bins)
    
    containing_array = []
    
    for i in range(len(arrayvals) - 1):
        is_there = 0
        for k in range(len(impact_array)):
            if impact_array[k] > arrayvals[i] and impact_array[k] <= arrayvals[i+1]:
                containing_array.append(second_array[k])
                is_there = 1
        if is_there == 0:
            containing_array.append(np.zeros(len(impact_array)))
    for k in range(len(impact_array)):
        if impact_array[k] > arrayvals[-1]:
            containing_array.append(np.zeros(len(impact_array)))
    
    return containing_array
        
        
    

In [78]:
def convert_to_1darrays(impactparameters = None, secondarray = None):
    new_impactparameters = []
    new_secondarray = []
    
    for i in range(len(impactparameters)):
        for k in range(len(secondarray[i])):
            new_impactparameters.append(impactparameters[i])
            
    for i in range(len(secondarray)):
        for k in range(len(secondarray[i])):
            new_secondarray.append(secondarray[i][k])
            
    return new_impactparameters, new_secondarray
        
    